In [ ]:
from __future__ import absolute_import
from __future__ import print_function

## TPU Check

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

TensorFlow 2.x selected.
Tensorflow version 2.1.0
Running on TPU  ['10.99.172.2:8470']
INFO:tensorflow:Initializing the TPU system: 10.99.172.2:8470


INFO:tensorflow:Initializing the TPU system: 10.99.172.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


## Some functions we might need (Siamese)

In [ ]:
import re
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf

import random
from keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing import image


def euclidean_distance(vects):#vects
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1 #1.25
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

Using TensorFlow backend.


## Model Architecture

In [ ]:
!pip install efficientnet

In [ ]:
#from tensorflow.keras.applications.xception import Xception
import efficientnet.tfkeras as efn 


"""
def create_model():
  input_shape=(313,214,3)

  xception = Xception(weights='imagenet', include_top=False,input_shape=input_shape,pooling='avg')

  # network definition
  base_network = xception

  input_a = Input(shape=input_shape)
  input_b = Input(shape=input_shape)

  # because we re-use the same instance `base_network`,
  # the weights of the network
  # will be shared across the two branches
  processed_a = base_network(input_a)
  processed_b = base_network(input_b)

  distance = Lambda(euclidean_distance,
                    output_shape=eucl_dist_output_shape)([processed_a, processed_b])

  return Model([input_a, input_b], distance)

"""

def create_model():
    input_shape=(317,232,3)

    effNet = efn.EfficientNetB4(weights= None, include_top=False, input_shape=input_shape, pooling="avg")  # or weights='imagenet' , 'noisy-student'

    #xception = Xception(weights='imagenet', include_top=False,input_shape=input_shape,pooling='avg')
    
        
    
    # network definition
    base_network = effNet #xception

    

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    distance = Lambda(euclidean_distance,
                    output_shape=eucl_dist_output_shape)([processed_a, processed_b])

    return Model([input_a, input_b], distance)


### Network Surgery 
Head Layer for classification  with EfficientNet (**post Siamese training**)

In [ ]:
class FCHeadNet:
    @staticmethod
    def build(baseModel, classes, D):
        #Initialize the hed model that will be placed on top of the base, then add a FC layer
        headModel = baseModel.output
        headModel = Flatten(name = "flatten")(headModel)
        headModel = Dense(D, activation = "relu")(headModel)
        headModel = Dropout(0.5)(headModel)

        # Add a softmax layer (in our case logistic should be the same)
        headModel = Dense(classes, activation = "softmax")(headModel)

        return headModel

## Accessing GS Bucket

In [ ]:
### Configuring our gs bucket (cloud storage)

# Set your own project id here
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = 'deepfakechallenge-269819'
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [ ]:
bucket_name = "deepfake_mljc_bucket"
gcs_pattern = 'gs://deepfake_mljc_bucket/TFRecs/Training/*.tfrecords'
gcs_valid_pattern = 'gs://deepfake_mljc_bucket/TFRecs/Validation/df0.tfrecords'

### Checkpointing our Model

Every 1000 iterations

In [ ]:
# It saves the model every 1000 epochs --> to be lately put in the tpu_model as soon as it works 
import os
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "gs://deepfake_mljc_bucket/Checkpoints/cp_tpu.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = ModelCheckpoint(checkpoint_path, save_freq= 1000,
                                                 verbose=1)


In [ ]:
!gsutil ls gs://deepfake_mljc_bucket/

gs://deepfake_mljc_bucket/Audio/
gs://deepfake_mljc_bucket/Checkpoints/
gs://deepfake_mljc_bucket/FacesDataframes/
gs://deepfake_mljc_bucket/TFRecs/


## TFRecords 

In [ ]:
#Training Set
filenames = tf.io.gfile.glob(gcs_pattern)
#Validation Set
validation_fns = tf.io.gfile.glob(gcs_valid_pattern)

In [ ]:
filenames[35]

'gs://deepfake_mljc_bucket/TFRecs/Training/df41.tfrecords'

### Training Set

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False

#filenames = tf.io.gfile.glob(FILENAME_PATTERN)
dataset = tf.data.TFRecordDataset(filenames[35], num_parallel_reads=AUTO)
#dataset = dataset.with_options(ignore_order)

BATCH_SIZE = 128 #16 * tpu_strategy.num_replicas_in_sync #256
PREFETCH_SIZE= 256 #32 * tpu_strategy.num_replicas_in_sync #512 

def read_tfrecord(data):
    image_feature_description = {

      'filename_1': tf.io.FixedLenFeature([], tf.string),
      'filename_2': tf.io.FixedLenFeature([], tf.string),

      #'width': tf.io.FixedLenFeature([], tf.int64),
      #'height': tf.io.FixedLenFeature([], tf.int64),

      #'channels': tf.io.FixedLenFeature([], tf.int64),
      'image_1': tf.io.FixedLenFeature([], tf.string),
      'image_2': tf.io.FixedLenFeature([], tf.string),
      'label': tf.io.FixedLenFeature([], tf.float32),
      }

    tf_record = tf.io.parse_single_example(data, image_feature_description)
    img_height = 317
    img_width = 232
    img_size = (img_height, img_width)

    # ISSUE WITH TF.IMAGE.RESIZE
    # https://hackernoon.com/how-tensorflows-tf-image-resize-stole-60-days-of-my-life-aba5eb093f35 

    image_1 = tf.image.decode_png(tf_record["image_1"], channels = 3)
    # Keep it commented if you're using padded and cropped data since the dimension is the default one
    image_1 = tf.image.resize(image_1, img_size)
    image_1 = tf.cast(image_1, tf.float32)
    #Teoricamente non dovrebbe servire nemmeno la standardization, siccome nella creazione dei tf records si è usato tf.image.decode_png --> i valori dei pixel sono float in [0,1]
    image_1 = tf.image.per_image_standardization(image_1)


    image_2 = tf.image.decode_png(tf_record["image_2"], channels = 3,)

    # Keep it commented if you're using padded and cropped data since the dimension is the default one
    image_2 = tf.image.resize(image_2, img_size)
    image_2 = tf.cast(image_2, tf.float32)
    image_2 = tf.image.per_image_standardization(image_2)




    label = tf_record["label"]
    #label = tf.cast(label, tf.int32)  

    #def _parse_image_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    # return tf.io.parse_single_example(example_proto, image_feature_description)


    return (image_1, image_2), label

dataset = dataset.map(read_tfrecord)
dataset = dataset.shuffle(BATCH_SIZE*16, reshuffle_each_iteration=True) 

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(PREFETCH_SIZE)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [ ]:
dataset

<PrefetchDataset shapes: (((128, 317, 232, 3), (128, 317, 232, 3)), (128,)), types: ((tf.float32, tf.float32), tf.float32)>

### Validation Set

In [ ]:
valid_set = tf.data.TFRecordDataset(validation_fns, num_parallel_reads=AUTO)
valid_set= valid_set.map(read_tfrecord)
valid_set = valid_set.shuffle(BATCH_SIZE*16, reshuffle_each_iteration=True) 

valid_set = valid_set.batch(BATCH_SIZE, drop_remainder=True).prefetch(PREFETCH_SIZE)

In [ ]:
valid_set

<PrefetchDataset shapes: (((128, 317, 232, 3), (128, 317, 232, 3)), (128,)), types: ((tf.float32, tf.float32), tf.float32)>

## TPU Training

In [ ]:
#learning_rate = 0.1

#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = learning_rate, decay_steps= (800), decay_rate= 1e-4)
#opt = tf.keras.optimizers.SGD(learning_rate= lr_schedule, momentum = 0.9)
opt = tf.keras.optimizers.SGD(learning_rate= 0.01) #, momentum = 0.9, nesterov= True)

#checkpoint_path = "gs://deepfake_mljc_bucket/Checkpoints/cp_tpu1.ckpt"
checkpoint_path = "gs://deepfake_mljc_bucket/Checkpoints/Margin1/cp_tpu1.ckpt"
with tpu_strategy.scope():
    model = create_model()
    
    for layer in  model.layers[2].layers[:-326]:
      layer.trainable = False 
    #for layer in model.layers[2].layers[:-237]:
      #layer.trainable = False 

    model.compile(
    optimizer = opt,
    loss=contrastive_loss,
    metrics=[accuracy]) 

    model.load_weights(checkpoint_path)
    
# class_weight = {'0.0': 0.27586, '1.0': 0.72414}

In [ ]:
history = model.fit(dataset, epochs= 5, shuffle = True)#, validation_data= valid_set, validation_freq=5) 

Epoch 1/5
666/666 [==============================] - 1353s 2s/step - loss: 0.1522 - accuracy: 0.7751
Epoch 2/5
  1/666 [..............................] - ETA: 19:41

UnavailableError: ignored

In [ ]:
checkpoint_path = "gs://deepfake_mljc_bucket/Checkpoints/Margin1/cp_tpu1.ckpt"
model.save_weights(checkpoint_path, overwrite= True)